# Evaluate

This notebook evaluates the network trained previous notebooks and analyzes the results.

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%cd ..
%load_ext autoreload
%autoreload 2

C:\Users\Minjae\Desktop\EEG_Project


In [3]:
# Load some packages
import os
import sys
import pickle
from copy import deepcopy
import hydra
from omegaconf import OmegaConf
from collections import OrderedDict

import numpy as np
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

import pprint
import wandb
import matplotlib
import matplotlib.pyplot as plt

# custom package
from datasets.caueeg_script import build_dataset_for_train
import models
from train.evaluate import check_accuracy
from train.evaluate import check_accuracy_extended
from train.evaluate import check_accuracy_extended_debug
from train.evaluate import check_accuracy_multicrop
from train.evaluate import check_accuracy_multicrop_extended
from train.evaluate import calculate_class_wise_metrics
from train.visualize import draw_roc_curve
from train.visualize import draw_confusion
from train.visualize import draw_class_wise_metrics
from train.visualize import draw_error_table
from train.visualize import annotate_heatmap

In [4]:
print('PyTorch version:', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.') 

PyTorch version: 1.11.0+cu113
cuda is available.


-----

## List up the models to check accuracy

In [5]:
# model_names = ['cix5o8ut', '76efnucq', '6a1tnz3r']
model_names = [
'3jmkkwbc',
'cygczhuk',
'ph0mix3b',
'3cy4eb97',
'1cmzq9xq',
'3t0s53ki',
'52v2mp7f',
'1o7cz2rs',
'86s3czvy',
'gma3iwco',
'3pg95uf1',
'2s1700lg',
'1nq1zwxk',
'1xhv4ae6',
'13i117jq',
'1242j086',
'dejxbfxu',
'2z7is3m0',
'2h30v12p',
'3c9rbdxu',
'2615y2jm',
'2452kedi',
'2k8xomy6',
'2bw00w05',
'3874otvp',
]
model_pool = []

for model_name in model_names:
    path = os.path.join(r'E:\CAUEEG\checkpoint', model_name, 'checkpoint.pt')
    try:
        ckpt = torch.load(path, map_location=device)
        model_pool.append({'name': model_name, 'path': path})
    except Exception as e:
        print(e)
        print(f'- checkpoint cannot be opened: {path}')
        
pprint.pprint([model_dict['name'] for model_dict in model_pool])

['3jmkkwbc',
 'cygczhuk',
 'ph0mix3b',
 '3cy4eb97',
 '1cmzq9xq',
 '3t0s53ki',
 '52v2mp7f',
 '1o7cz2rs',
 '86s3czvy',
 'gma3iwco',
 '3pg95uf1',
 '2s1700lg',
 '1nq1zwxk',
 '1xhv4ae6',
 '13i117jq',
 '1242j086',
 'dejxbfxu',
 '2z7is3m0',
 '2h30v12p',
 '3c9rbdxu',
 '2615y2jm',
 '2452kedi',
 '2k8xomy6',
 '2bw00w05',
 '3874otvp']


In [6]:
# model_pool = []

# api = wandb.Api()
# runs = api.runs('ipis-mjkim/caueeg-task2-ablation')

# for run in runs:
#     path = os.path.join(r'E:\CAUEEG\checkpoint', run.name, 'checkpoint.pt')
#     try:
#         ckpt = torch.load(path, map_location=device)
#         model_pool.append({'name': run.name, 'path': path})
#     except Exception as e:
#         print(e)
#         print(f'- {run.name}\'s checkpoint cannot be opened: {path}')
        
# pprint.pprint([model_dict['name'] for model_dict in model_pool])

---

## Configurations

In [7]:
base_repeat = 200
verbose = False

-----

## Load and check accuracy

In [8]:
for model_dict in model_pool:
    # load and parse the checkpoint
    ckpt = torch.load(model_dict['path'], map_location=device)
    model_state = ckpt['model_state']
    config = ckpt['config']
    optimizer = ckpt['optimizer_state']
    scheduler = ckpt['scheduler_state']
    
    print('- checking for', model_dict['name'], config['model'], '...')
    
    # initiate the model
    if '_target_' in config:
        model = hydra.utils.instantiate(config).to(device)
    elif type(config['generator']) is str:
        config['generator'] = getattr(models, config['generator'].split('.')[-1])
        if 'block' in config:
            config['block'] = getattr(models, config['block'].split('.')[-1])
        model = config['generator'](**config).to(device)
    else:
        if 'block' in config:
            if config['block'] == models.resnet_1d.BottleneckBlock1D:
                config['block'] = 'bottleneck'
            elif config['block'] == models.resnet_2d.Bottleneck2D:
                config['block'] = 'bottleneck'
            elif config['block'] == models.resnet_1d.BasicBlock1D:
                config['block'] = 'basic'
            elif config['block'] == models.resnet_2d.BasicBlock2D:
                config['block'] = 'basic'
                
        model = config['generator'](**config).to(device)
    
    if config.get('ddp', False):
        model_state_ddp = deepcopy(model_state)
        model_state = OrderedDict()
        for k, v in model_state_ddp.items():
            name = k[7:]  # remove 'module.' of DataParallel/DistributedDataParallel
            model_state[name] = v
    
    model.load_state_dict(model_state)
    
    # reconfigure
    config.pop('cwd', 0)
    config['ddp'] = False
    config['crop_timing_analysis'] = False
    config['eval'] = True
    config['device'] = device
    
    repeat = round(base_repeat / config['crop_multiple'])
    model_dict['repeat'] = repeat
    model_dict['crop_multiple'] = config['crop_multiple']
    model_dict['test_crop_multiple'] = config['test_crop_multiple']    
    
    # build dataset
    _ = build_dataset_for_train(config, verbose=verbose)
    train_loader = _[0]
    val_loader = _[1]
    test_loader = _[2]
    multicrop_test_loader = _[3]
    _, multicrop_val_dataset = load_caueeg_task_split(dataset_path=config['dataset_path'],
                                                      task=config['task'],
                                                      split='val',
                                                      load_event=config['load_event'],
                                                      file_format=config['file_format'],
                                                      transform=config['transform_multicrop'],
                                                      verbose=verbose)
    multicrop_val_loader = DataLoader(multicrop_val_dataset,
                                       batch_size=config['multi_batch_size'],
                                       shuffle=False,
                                       drop_last=False,
                                       num_workers=0,
                                       pin_memory=True,
                                       collate_fn=eeg_collate_fn)
    
    # train accuracy
    train_acc = check_accuracy(model, train_loader, 
                               config['preprocess_test'], config, repeat=repeat)
    model_dict['Train Accuracy'] = train_acc
    
    # val accuracy
    val_acc = check_accuracy(model, val_loader, 
                             config['preprocess_test'], config, repeat=repeat)
    model_dict['Validation Accuracy'] = val_acc

    # Multi-crop val accuracy
    _ = check_accuracy_multicrop_extended(model, multicrop_val_loader, 
                                          config['preprocess_test'], config, repeat=repeat)
    model_dict['Multi-Crop Val Accuracy'] = _[0]
    model_dict['Multi-Crop Val Throughput'] = _[4]
    multi_val_class_wise_metrics = calculate_class_wise_metrics(_[3])
    
    for k, v in multi_val_class_wise_metrics.items():
        for c in range(config['out_dims']):
            c_name = config['class_label_to_name'][c]
            model_dict[f'Multi-Crop Val {k} ({c_name})'] = multi_val_class_wise_metrics[k][c]
    
    
    # Test accuracy
    test_acc = check_accuracy(model, test_loader, 
                       config['preprocess_test'], config, repeat=repeat)
    model_dict['Test Accuracy'] = test_acc

    # Multi-crop test accuracy
    _ = check_accuracy_multicrop_extended(model, multicrop_test_loader, 
                                          config['preprocess_test'], config, repeat=repeat)  
    model_dict['Multi-Crop Test Accuracy'] = _[0]
    model_dict['Multi-Crop Test Throughput'] = _[4]
    multi_test_class_wise_metrics = calculate_class_wise_metrics(_[3])
    
    for k, v in multi_test_class_wise_metrics.items():
        for c in range(config['out_dims']):
            c_name = config['class_label_to_name'][c]
            model_dict[f'Multi-Crop Test {k} ({c_name})'] = multi_test_class_wise_metrics[k][c]

- checking for 3jmkkwbc 1D-ResNet-18 ...
- checking for cygczhuk 1D-ResNet-18 ...
- checking for ph0mix3b 1D-ResNet-18 ...
- checking for 3cy4eb97 1D-ResNet-18 ...
- checking for 1cmzq9xq 1D-ResNet-18 ...
- checking for 3t0s53ki 1D-ResNet-18 ...
- checking for 52v2mp7f 1D-ResNet-18 ...
- checking for 1o7cz2rs 1D-ResNet-18 ...
- checking for 86s3czvy 1D-ResNet-18 ...
- checking for gma3iwco 1D-ResNet-18 ...
- checking for 3pg95uf1 1D-ResNet-18 ...
- checking for 2s1700lg 1D-ResNet-18 ...
- checking for 1nq1zwxk 1D-ResNet-18 ...
- checking for 1xhv4ae6 1D-ResNet-18 ...
- checking for 13i117jq 1D-ResNet-18 ...
- checking for 1242j086 1D-ResNet-18 ...
- checking for dejxbfxu 1D-ResNet-18 ...
- checking for 2z7is3m0 1D-ResNet-18 ...
- checking for 2h30v12p 1D-ResNet-18 ...
- checking for 3c9rbdxu 1D-ResNet-18 ...
- checking for 2615y2jm 1D-ResNet-18 ...
- checking for 2452kedi 1D-ResNet-18 ...
- checking for 2k8xomy6 1D-ResNet-18 ...
- checking for 2bw00w05 1D-ResNet-18 ...
- checking for 3

In [9]:
pprint.pprint(model_pool)

[{'Multi-Crop Test Accuracy': 61.6271186440678,
  'Multi-Crop Test Accuracy (Dementia)': 0.7671186440677966,
  'Multi-Crop Test Accuracy (MCI)': 0.6383050847457628,
  'Multi-Crop Test Accuracy (Normal)': 0.8271186440677966,
  'Multi-Crop Test F1-score (Dementia)': 0.524896265560166,
  'Multi-Crop Test F1-score (MCI)': 0.5078413284132842,
  'Multi-Crop Test F1-score (Normal)': 0.7769028871391076,
  'Multi-Crop Test Precision (Dementia)': 0.5655737704918032,
  'Multi-Crop Test Precision (MCI)': 0.4816272965879265,
  'Multi-Crop Test Precision (Normal)': 0.7816901408450704,
  'Multi-Crop Test Recall (Dementia)': 0.4896774193548387,
  'Multi-Crop Test Recall (MCI)': 0.5370731707317074,
  'Multi-Crop Test Recall (Normal)': 0.7721739130434783,
  'Multi-Crop Test Sensitivity (Dementia)': 0.4896774193548387,
  'Multi-Crop Test Sensitivity (MCI)': 0.5370731707317074,
  'Multi-Crop Test Sensitivity (Normal)': 0.7721739130434783,
  'Multi-Crop Test Specificity (Dementia)': 0.8659770114942529,
  '

In [10]:
for model_dict in model_pool:
    # load and parse the checkpoint
    ckpt = torch.load(model_dict['path'], map_location=device)
    model_state = ckpt['model_state']
    config = ckpt['config']
        
    model_dict['model'] = config['model']
    model_dict['num_params'] = config.get('num_params', '???')
    model_dict['model size (MiB)'] = sys.getsizeof(pickle.dumps(model_state)) / (1024 * 1024)
    # torch.save(model_state, 'temptemptemp.pt')
    
    model_dict['seq_length'] = config['seq_length']
    model_dict['use_age'] = config['use_age']
    model_dict['photic'] = config['photic']
    model_dict['EKG'] = config['EKG']

    model_dict['awgn'] = config.get('awgn', 0)
    model_dict['awgn_age'] = config.get('awgn_age', 0)
    model_dict['mgn'] = config.get('mgn', 0)
    model_dict['mixup'] = config.get('mixup', 0)
    model_dict['dropout'] = config.get('dropout', 0)
    model_dict['weight_decay'] = config.get('weight_decay', '???')
    model_dict['fc_stages'] = config.get('fc_stages', 1)
    
    model_dict['minibatch'] = round(config['minibatch'])
    model_dict['total_samples'] = round(config.get('total_samples', config['iterations'] * config['minibatch']))
    model_dict['base_lr'] = config.get('base_lr', config.get('LR', '???'))
    model_dict['lr_scheduler_type'] = config.get('lr_scheduler_type', 'constant_with_decay')
    model_dict['warmup_steps'] = config.get('warmup_steps', '???')
    model_dict['seed'] = config.get('seed', '???')

In [11]:
pd.DataFrame(model_pool)

,name,path,repeat,crop_multiple,test_crop_multiple,Train Accuracy,Validation Accuracy,Multi-Crop Val Accuracy,Multi-Crop Val Throughput,Multi-Crop Val Accuracy (Normal),...,mixup,dropout,weight_decay,fc_stages,minibatch,total_samples,base_lr,lr_scheduler_type,warmup_steps,seed
0,3jmkkwbc,E:\CAUEEG\checkpoint\3jmkkwbc\checkpoint.pt,50,4,8,99.347656,63.268908,65.159664,1159.526581,0.772941,...,0.2,0.3,0.043947,3,256,100000000,0.000469,cosine_decay_with_warmup_half,19531,0
1,cygczhuk,E:\CAUEEG\checkpoint\cygczhuk\checkpoint.pt,50,4,8,99.670201,56.109244,57.008403,1017.844120,0.688739,...,0.0,0.0,0.043947,1,256,100000000,0.000469,cosine_decay_with_warmup_half,19531,1
2,ph0mix3b,E:\CAUEEG\checkpoint\ph0mix3b\checkpoint.pt,50,4,8,100.000000,57.815126,59.176471,1205.701318,0.707563,...,0.0,0.0,0.043947,3,512,100000000,0.000339,cosine_decay_with_warmup_half,9766,1
3,3cy4eb97,E:\CAUEEG\checkpoint\3cy4eb97\checkpoint.pt,50,4,8,100.000000,56.739496,56.974790,979.456526,0.716471,...,0.0,0.3,0.043947,3,256,100000000,0.000238,cosine_decay_with_warmup_half,19531,1
4,1cmzq9xq,E:\CAUEEG\checkpoint\1cmzq9xq\checkpoint.pt,50,4,8,99.987723,61.991597,65.075630,1206.630627,0.737311,...,0.2,0.3,0.043947,3,512,100000000,0.000476,cosine_decay_with_warmup_half,9766,1
5,3t0s53ki,E:\CAUEEG\checkpoint\3t0s53ki\checkpoint.pt,50,4,8,99.682478,59.512605,64.134454,1045.852356,0.759160,...,0.0,0.0,0.043947,1,256,100000000,0.000469,cosine_decay_with_warmup_half,19531,0
6,52v2mp7f,E:\CAUEEG\checkpoint\52v2mp7f\checkpoint.pt,50,4,8,99.746652,55.403361,57.764706,470.837463,0.665546,...,0.0,0.0,0.043947,1,512,100000000,0.000469,cosine_decay_with_warmup_half,9766,0
7,1o7cz2rs,E:\CAUEEG\checkpoint\1o7cz2rs\checkpoint.pt,50,4,8,100.000000,60.016807,61.714286,1255.282130,0.717479,...,0.0,0.0,0.043947,1,512,100000000,0.000469,cosine_decay_with_warmup_half,9766,0
8,86s3czvy,E:\CAUEEG\checkpoint\86s3czvy\checkpoint.pt,50,4,8,99.782366,56.630252,56.285714,1014.395209,0.693445,...,0.0,0.0,0.043947,3,256,100000000,0.000469,cosine_decay_with_warmup_half,19531,0
9,gma3iwco,E:\CAUEEG\checkpoint\gma3iwco\checkpoint.pt,50,4,8,100.000000,51.319328,52.941176,1245.034016,0.655630,...,0.0,0.0,0.043947,1,512,100000000,0.000339,cosine_decay_with_warmup_half,9766,1


In [12]:
pd.DataFrame(model_pool).to_csv('local/output/caueeg-task2-ablations-summary.csv')